## Federated Learning with one corrupted node: trimmed mean
Using one balanced and one unbalanced dataset with one corrupted node (5%, 25% and 50% corrupted samples) to test different aggregation functions and determine the more robust one. 

In [8]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [9]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [10]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [11]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [12]:
def aggregate(grad_list, trim_num = 1):
    # Calculate 20% trimmed mean -> stay with 3 closer nodes 
    # Initialize the list to store trimmed mean gradients
    trim_mean_grad = []
    
    # Iterate over the gradients coordinate-wise
    for layer_grads in zip(*grad_list):
        # Stack gradients for the current layer to shape (num_nodes, ...)
        stacked_grads = tf.stack(layer_grads, axis=0)

        # Sort the gradients along the node axis
        sorted_grads = tf.sort(stacked_grads, axis=0)

        # Trim the top and bottom trim_count gradients
        trimmed_grads = sorted_grads[trim_num:num_nodes - trim_num]

        # Calculate the mean of the remaining gradients
        trimmed_mean = tf.reduce_mean(trimmed_grads, axis=0)

        # Append the trimmed mean gradient for the current layer
        trim_mean_grad.append(trimmed_mean)
    
    
    return trim_mean_grad

In [13]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [14]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

#### 5A 5% 

In [15]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3687049449.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3687049449.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3687049449.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3687049449.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option o

In [16]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [17]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Atrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Atrim.hdf5')

Epoch 1/5
31/31 [==============================] - 4s 144ms/step - loss: 0.4719 - accuracy: 0.8668 - val_loss: 0.2438 - val_accuracy: 0.9240
Epoch 2/5
31/31 [==============================] - 5s 151ms/step - loss: 0.1922 - accuracy: 0.9522 - val_loss: 0.1810 - val_accuracy: 0.9600
Epoch 3/5
31/31 [==============================] - 5s 147ms/step - loss: 0.1672 - accuracy: 0.9643 - val_loss: 0.1626 - val_accuracy: 0.9629
Epoch 4/5
31/31 [==============================] - 5s 161ms/step - loss: 0.1533 - accuracy: 0.9668 - val_loss: 0.1532 - val_accuracy: 0.9651
Epoch 5/5
31/31 [==============================] - 5s 156ms/step - loss: 0.1459 - accuracy: 0.9676 - val_loss: 0.1484 - val_accuracy: 0.9657
Epoch 1/5
31/31 [==============================] - 5s 146ms/step - loss: 0.0351 - accuracy: 0.9898 - val_loss: 0.5027 - val_accuracy: 0.6757
Epoch 2/5
31/31 [==============================] - 5s 160ms/step - loss: 0.0236 - accuracy: 0.9908 - val_loss: 0.3351 - val_accuracy: 0.7642
Epoch 3/5
31/

In [22]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.042285747826099396, 0.2616843283176422], [0.08800294250249863, 0.54937344789505], [0.06275994330644608, 0.3403714597225189]]
Accuracy for iterations:  [[0.9788775444030762, 0.8806401491165161], [0.967410683631897, 0.7499818205833435], [0.972859263420105, 0.828798234462738]]
F1 for iterations:  [[0.9788615259035282, 0.8777836036028267], [0.9673305136426964, 0.7251205814396124], [0.9728185377982685, 0.8206576736913368]]
Precision for iterations:  [[0.979084447975743, 0.8949858527993769], [0.9688303261334784, 0.8169439479915453], [0.9735154730730012, 0.8608948198651091]]
Recall for iterations:  [[0.9788775599637735, 0.88064012206641], [0.9674106751584913, 0.7499818353556638], [0.9728592713780713, 0.8287982271307128]]


#### 5B 10%

In [23]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/2463029928.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/2463029928.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/2463029928.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/2463029928.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [24]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [25]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Atrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [26]:
global_model = build_model((24,1,1))

In [27]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [28]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [29]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Atrim.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 164ms/step - loss: 0.4948 - accuracy: 0.8260 - val_loss: 0.2943 - val_accuracy: 0.9043
Epoch 2/5
31/31 [==============================] - 5s 160ms/step - loss: 0.2646 - accuracy: 0.9289 - val_loss: 0.2453 - val_accuracy: 0.9402
Epoch 3/5
31/31 [==============================] - 5s 159ms/step - loss: 0.2428 - accuracy: 0.9406 - val_loss: 0.2341 - val_accuracy: 0.9414
Epoch 4/5
31/31 [==============================] - 5s 158ms/step - loss: 0.2318 - accuracy: 0.9418 - val_loss: 0.2255 - val_accuracy: 0.9426
Epoch 5/5
31/31 [==============================] - 5s 162ms/step - loss: 0.2252 - accuracy: 0.9425 - val_loss: 0.2207 - val_accuracy: 0.9428
Epoch 1/5
31/31 [==============================] - 5s 162ms/step - loss: 0.0373 - accuracy: 0.9897 - val_loss: 0.4601 - val_accuracy: 0.7144
Epoch 2/5
31/31 [==============================] - 5s 163ms/step - loss: 0.0236 - accuracy: 0.9905 - val_loss: 0.4199 - val_accuracy: 0.6780
Epoch 3/5
31/

In [30]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05028942599892616, 0.3205834925174713], [0.06852910667657852, 0.38822370767593384], [0.076668880879879, 0.37169456481933594]]
Accuracy for iterations:  [[0.9730053544044495, 0.8474169969558716], [0.9710990786552429, 0.8078362345695496], [0.9700326919555664, 0.8056201338768005]]
F1 for iterations:  [[0.972967334914303, 0.841549289072722], [0.9710459029564532, 0.7964318949359777], [0.9699702812954014, 0.7937612346782192]]
Precision for iterations:  [[0.9735953007015596, 0.8728276569160942], [0.9720083999680239, 0.8484449321571627], [0.9711383790561191, 0.8475302701275563]]
Recall for iterations:  [[0.9730053463437436, 0.8474169875753833], [0.971099068041719, 0.8078362275666643], [0.9700327207923106, 0.80562014095764]]


#### 5B 25%

In [31]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3855986297.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3855986297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3855986297.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12168/3855986297.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [32]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [33]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Atrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [34]:
global_model = build_model((24,1,1))

In [35]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [36]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [37]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Atrim.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 158ms/step - loss: 0.4922 - accuracy: 0.8231 - val_loss: 0.2886 - val_accuracy: 0.8563
Epoch 2/5
31/31 [==============================] - 5s 157ms/step - loss: 0.1797 - accuracy: 0.9435 - val_loss: 0.1261 - val_accuracy: 0.9748
Epoch 3/5
31/31 [==============================] - 5s 157ms/step - loss: 0.1197 - accuracy: 0.9768 - val_loss: 0.1135 - val_accuracy: 0.9779
Epoch 4/5
31/31 [==============================] - 5s 156ms/step - loss: 0.1084 - accuracy: 0.9783 - val_loss: 0.1067 - val_accuracy: 0.9774
Epoch 5/5
31/31 [==============================] - 5s 159ms/step - loss: 0.1011 - accuracy: 0.9788 - val_loss: 0.1009 - val_accuracy: 0.9787
Epoch 1/5
31/31 [==============================] - 5s 153ms/step - loss: 0.0295 - accuracy: 0.9899 - val_loss: 0.4507 - val_accuracy: 0.7028
Epoch 2/5
31/31 [==============================] - 5s 162ms/step - loss: 0.0228 - accuracy: 0.9909 - val_loss: 0.2226 - val_accuracy: 0.8975
Epoch 3/5
31/

In [38]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.03547447919845581, 0.24176229536533356], [0.06090780347585678, 0.5759752988815308], [0.09472198039293289, 0.8302773833274841]]
Accuracy for iterations:  [[0.98721843957901, 0.8914117813110352], [0.9727716445922852, 0.7192472815513611], [0.9665049910545349, 0.6865509152412415]]
F1 for iterations:  [[0.9872179163877111, 0.8891381357650477], [0.9727271160510258, 0.6836903376510435], [0.9664137908651006, 0.6358010262225898]]
Precision for iterations:  [[0.9872183662570505, 0.9036799999619477], [0.9735276561088202, 0.8023977883403604], [0.968161154710334, 0.7898784531411435]]
Recall for iterations:  [[0.9872184405036665, 0.8914117561578144], [0.9727716263986678, 0.7192472571387052], [0.9665050103713225, 0.6865508973334302]]


#### 5B 5% 

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/3503541895.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/3503541895.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/3503541895.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')


In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Btrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Btrim.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 179ms/step - loss: 0.4804 - accuracy: 0.8230 - val_loss: 0.2525 - val_accuracy: 0.9144
Epoch 2/5
31/31 [==============================] - 6s 178ms/step - loss: 0.2023 - accuracy: 0.9473 - val_loss: 0.1742 - val_accuracy: 0.9605
Epoch 3/5
31/31 [==============================] - 5s 174ms/step - loss: 0.1699 - accuracy: 0.9635 - val_loss: 0.1602 - val_accuracy: 0.9642
Epoch 4/5
31/31 [==============================] - 6s 193ms/step - loss: 0.1549 - accuracy: 0.9659 - val_loss: 0.1512 - val_accuracy: 0.9658
Epoch 5/5
31/31 [==============================] - 6s 188ms/step - loss: 0.1470 - accuracy: 0.9674 - val_loss: 0.1466 - val_accuracy: 0.9660 loss: 0.147
Epoch 1/5
31/31 [==============================] - 5s 171ms/step - loss: 0.0357 - accuracy: 0.9899 - val_loss: 0.4334 - val_accuracy: 0.7160
Epoch 2/5
31/31 [==============================] - 5s 168ms/step - loss: 0.0233 - accuracy: 0.9907 - val_loss: 0.3948 - val_accuracy: 0.6877
E

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.12244821339845657, 0.2528262138366699], [0.08950452506542206, 0.24348971247673035], [0.07490818202495575, 0.27225756645202637]]
Accuracy for iterations:  [[0.924260139465332, 0.8514131903648376], [0.9445280432701111, 0.8602412343025208], [0.9593692421913147, 0.8637470006942749]]
F1 for iterations:  [[0.9235397984106394, 0.8459784484330178], [0.944217950472414, 0.8557242476851611], [0.959231231785833, 0.8594891071087266]]
Precision for iterations:  [[0.9328015488315454, 0.8754066001815818], [0.9489238365189864, 0.8809455090187539], [0.9616180683113731, 0.8837468987190035]]
Recall for iterations:  [[0.9242601302988693, 0.8514132093293614], [0.9445280317859125, 0.8602412264767856], [0.9593692482982267, 0.8637470028336846]]


#### 5B 10%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/2097221778.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/2097221778.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/2097221778.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_12308/2097221778.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Btrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Btrim.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 253ms/step - loss: 0.4764 - accuracy: 0.8579 - val_loss: 0.2773 - val_accuracy: 0.9208
Epoch 2/5
31/31 [==============================] - 6s 198ms/step - loss: 0.2672 - accuracy: 0.9289 - val_loss: 0.2492 - val_accuracy: 0.9397
Epoch 3/5
31/31 [==============================] - 6s 209ms/step - loss: 0.2447 - accuracy: 0.9394 - val_loss: 0.2319 - val_accuracy: 0.9415
Epoch 4/5
31/31 [==============================] - 7s 221ms/step - loss: 0.2310 - accuracy: 0.9418 - val_loss: 0.2224 - val_accuracy: 0.9431
Epoch 5/5
31/31 [==============================] - 7s 242ms/step - loss: 0.2236 - accuracy: 0.9424 - val_loss: 0.2177 - val_accuracy: 0.9433
Epoch 1/5
31/31 [==============================] - 7s 226ms/step - loss: 0.0369 - accuracy: 0.9902 - val_loss: 0.4042 - val_accuracy: 0.7557
Epoch 2/5
31/31 [==============================] - 7s 217ms/step - loss: 0.0237 - accuracy: 0.9911 - val_loss: 0.3248 - val_accuracy: 0.7816
Epoch 3/5
31/

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.122221939265728, 0.2280646115541458], [0.10179457068443298, 0.28196805715560913], [0.09244326502084732, 0.30414462089538574]]
Accuracy for iterations:  [[0.9255455732345581, 0.8877425193786621], [0.941372811794281, 0.8346290588378906], [0.9510868191719055, 0.838207483291626]]
F1 for iterations:  [[0.9248587934917004, 0.8852297838198837], [0.9410112177452631, 0.8272093187585741], [0.9508598994241823, 0.8312715046375951]]
Precision for iterations:  [[0.9337771859800958, 0.9011270863559564], [0.9463667913714228, 0.8648585340365701], [0.9545341635122602, 0.8668976370039558]]
Recall for iterations:  [[0.9255455899967864, 0.8877424980018891], [0.941372812527389, 0.8346290779626535], [0.9510867977446025, 0.8382075128968974]]


#### 5B 25%

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/3291834707.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/3291834707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/3291834707.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/3291834707.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Btrim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Btrim.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 158ms/step - loss: 0.5489 - accuracy: 0.7542 - val_loss: 0.4311 - val_accuracy: 0.8559
Epoch 2/5
31/31 [==============================] - 5s 152ms/step - loss: 0.4217 - accuracy: 0.8586 - val_loss: 0.4032 - val_accuracy: 0.8683
Epoch 3/5
31/31 [==============================] - 5s 153ms/step - loss: 0.4051 - accuracy: 0.8662 - val_loss: 0.3924 - val_accuracy: 0.8700
Epoch 4/5
31/31 [==============================] - 5s 150ms/step - loss: 0.3975 - accuracy: 0.8679 - val_loss: 0.3886 - val_accuracy: 0.8704
Epoch 5/5
31/31 [==============================] - 5s 149ms/step - loss: 0.3936 - accuracy: 0.8683 - val_loss: 0.3887 - val_accuracy: 0.8705
Epoch 1/5
31/31 [==============================] - 5s 160ms/step - loss: 0.0505 - accuracy: 0.9895 - val_loss: 0.5217 - val_accuracy: 0.6561
Epoch 2/5
31/31 [==============================] - 5s 158ms/step - loss: 0.0241 - accuracy: 0.9908 - val_loss: 0.4075 - val_accuracy: 0.6819
Epoch 3/5
31/

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.12718556821346283, 0.2506321370601654], [0.09431221336126328, 0.20777417719364166], [0.08693456649780273, 0.2302439659833908]]
Accuracy for iterations:  [[0.9239314794540405, 0.8514677286148071], [0.9384732246398926, 0.9015294909477234], [0.9468725323677063, 0.8750090599060059]]
F1 for iterations:  [[0.9232044215951977, 0.8462135463954481], [0.9380632083276941, 0.8997971859796177], [0.9465919401131463, 0.8716070442680398]]
Precision for iterations:  [[0.9325202875460346, 0.8742504965436585], [0.9439814188043357, 0.9113137972507154], [0.9509623857296089, 0.8920751366213381]]
Recall for iterations:  [[0.9239314616261065, 0.8514677032623701], [0.9384732244587922, 0.9015294630531134], [0.9468725349849543, 0.8750090823221681]]


#### 5C NODE 1 5%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/591709827.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')


In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C1trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1trim.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 207ms/step - loss: 0.6056 - accuracy: 0.7805 - val_loss: 0.4711 - val_accuracy: 0.7837
Epoch 2/5
16/16 [==============================] - 3s 195ms/step - loss: 0.3646 - accuracy: 0.8595 - val_loss: 0.2521 - val_accuracy: 0.9267
Epoch 3/5
16/16 [==============================] - 3s 185ms/step - loss: 0.2224 - accuracy: 0.9405 - val_loss: 0.1963 - val_accuracy: 0.9553
Epoch 4/5
16/16 [==============================] - 3s 185ms/step - loss: 0.1947 - accuracy: 0.9583 - val_loss: 0.1804 - val_accuracy: 0.9615
Epoch 5/5
16/16 [==============================] - 3s 191ms/step - loss: 0.1819 - accuracy: 0.9621 - val_loss: 0.1696 - val_accuracy: 0.9645
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0416 - accuracy: 0.9903 - val_loss: 0.6089 - val_accuracy: 0.7274
Epoch 2/5
14/14 [==============================] - 3s 196ms/step - loss: 0.0239 - accuracy: 0.9936 - val_loss: 0.8258 - val_accuracy: 0.7270
Epoch 3/5
14/

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.062733955681324, 0.12458465248346329], [0.06769371032714844, 0.15497343242168427], [0.08265060186386108, 0.15783391892910004]]
Accuracy for iterations:  [[0.9875178933143616, 0.975023627281189], [0.9873718023300171, 0.9530262351036072], [0.9821934700012207, 0.950610339641571]]
F1 for iterations:  [[0.987521807169782, 0.9750497780067887], [0.9873789847612794, 0.9531916411137277], [0.9822060764648011, 0.9507851310990553]]
Precision for iterations:  [[0.9875986220703395, 0.9752430062115834], [0.9876991976496031, 0.9575644361152994], [0.9828387569128129, 0.9552772476117818]]
Recall for iterations:  [[0.9875178941832948, 0.9750236140376372], [0.9873718192176225, 0.9530262297464216], [0.9821934616845365, 0.9506103320496985]]


#### 5C NODE 1 10%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/372464333.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')


In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C1trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1trim.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 229ms/step - loss: 0.6243 - accuracy: 0.6882 - val_loss: 0.5144 - val_accuracy: 0.7838
Epoch 2/5
16/16 [==============================] - 10s 615ms/step - loss: 0.4160 - accuracy: 0.8354 - val_loss: 0.3155 - val_accuracy: 0.9043
Epoch 3/5
16/16 [==============================] - 5s 295ms/step - loss: 0.2934 - accuracy: 0.9173 - val_loss: 0.2806 - val_accuracy: 0.9293
Epoch 4/5
16/16 [==============================] - 4s 252ms/step - loss: 0.2688 - accuracy: 0.9344 - val_loss: 0.2662 - val_accuracy: 0.9383
Epoch 5/5
16/16 [==============================] - 4s 243ms/step - loss: 0.2572 - accuracy: 0.9382 - val_loss: 0.2556 - val_accuracy: 0.9394
Epoch 1/5
14/14 [==============================] - 3s 232ms/step - loss: 0.0562 - accuracy: 0.9903 - val_loss: 0.6054 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 239ms/step - loss: 0.0287 - accuracy: 0.9936 - val_loss: 0.7423 - val_accuracy: 0.7270
Epoch 3/5
14

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08047237247228622, 0.15403608977794647], [0.05477147549390793, 0.10293388366699219], [0.0783243402838707, 0.20254896581172943]]
Accuracy for iterations:  [[0.9870796799659729, 0.9523723125457764], [0.990234911441803, 0.9816718697547913], [0.9847862720489502, 0.9284676313400269]]
F1 for iterations:  [[0.9870829419179444, 0.9521742472039973], [0.9902394206199777, 0.9817068548328882], [0.9847960923824791, 0.928724059622684]]
Precision for iterations:  [[0.9871314968459036, 0.9534397207559683], [0.9904291343131405, 0.9824032818165005], [0.98525858467648, 0.9384855711664822]]
Recall for iterations:  [[0.9870796692862778, 0.952372302550316], [0.9902348885448012, 0.9816718738647098], [0.9847862923252213, 0.9284676306037928]]


#### 5C NODE 1 25%

In [32]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/2868219575.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')


In [33]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [34]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C1trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [35]:
global_model = build_model((24,1,1))

In [36]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [37]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [38]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1trim.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 251ms/step - loss: 0.6313 - accuracy: 0.7003 - val_loss: 0.5432 - val_accuracy: 0.7077
Epoch 2/5
16/16 [==============================] - 4s 248ms/step - loss: 0.4903 - accuracy: 0.7938 - val_loss: 0.4393 - val_accuracy: 0.8516
Epoch 3/5
16/16 [==============================] - 4s 243ms/step - loss: 0.4341 - accuracy: 0.8548 - val_loss: 0.4181 - val_accuracy: 0.8648
Epoch 4/5
16/16 [==============================] - 4s 249ms/step - loss: 0.4210 - accuracy: 0.8630 - val_loss: 0.4070 - val_accuracy: 0.8681
Epoch 5/5
16/16 [==============================] - 4s 239ms/step - loss: 0.4132 - accuracy: 0.8647 - val_loss: 0.4011 - val_accuracy: 0.8686
Epoch 1/5
14/14 [==============================] - 4s 253ms/step - loss: 0.0767 - accuracy: 0.9907 - val_loss: 0.4384 - val_accuracy: 0.7274
Epoch 2/5
14/14 [==============================] - 4s 255ms/step - loss: 0.0307 - accuracy: 0.9936 - val_loss: 0.9639 - val_accuracy: 0.7258
Epoch 3/5
14/

In [39]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06338245421648026, 0.113584503531456], [0.08071856200695038, 0.1986856460571289], [0.1418112963438034, 0.37868085503578186]]
Accuracy for iterations:  [[0.9894095659255981, 0.9835428595542908], [0.9849250912666321, 0.9306837320327759], [0.960567057132721, 0.8610586524009705]]
F1 for iterations:  [[0.9894140573340214, 0.9835571912352438], [0.9849347229392563, 0.9309352354185929], [0.9605953312486784, 0.8607222475195819]]
Precision for iterations:  [[0.9895667626532325, 0.9836851099430577], [0.9853897561861975, 0.9401315485917465], [0.9636055899126356, 0.8944473410099415]]
Recall for iterations:  [[0.9894095649887522, 0.9835428322313449], [0.9849250635426101, 0.930683717212817], [0.9605670630167402, 0.8610586354719174]]


#### 5C NODE 3 5%

In [40]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16296/927562370.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')


In [41]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C3trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
global_model = build_model((24,1,1))

In [44]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

KeyboardInterrupt: 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3trim.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 168ms/step - loss: 0.5297 - accuracy: 0.7412 - val_loss: 1.7559 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 170ms/step - loss: 0.1924 - accuracy: 0.9607 - val_loss: 3.6784 - val_accuracy: 0.1332
Epoch 3/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0802 - accuracy: 0.9893 - val_loss: 3.6751 - val_accuracy: 0.1342
Epoch 4/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0515 - accuracy: 0.9893 - val_loss: 1.8339 - val_accuracy: 0.1357
Epoch 5/5
16/16 [==============================] - 3s 202ms/step - loss: 0.0374 - accuracy: 0.9899 - val_loss: 1.5210 - val_accuracy: 0.1622
Epoch 1/5
14/14 [==============================] - 3s 230ms/step - loss: 0.0231 - accuracy: 0.9949 - val_loss: 0.8074 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 197ms/step - loss: 0.0208 - accuracy: 0.9949 - val_loss: 0.5096 - val_accuracy: 0.7351
Epoch 3/5

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1084146797657013, 0.1123688668012619], [0.1162383109331131, 0.1426282525062561], [0.11805704981088638, 0.16804981231689453]]
Accuracy for iterations:  [[0.9879999160766602, 0.9727712273597717], [0.9890370965003967, 0.952953577041626], [0.9900742173194885, 0.9472498893737793]]
F1 for iterations:  [[0.988003964019539, 0.9727450833085427], [0.9890408833630392, 0.9527551368789061], [0.9900774749113563, 0.9469688561753138]]
Precision for iterations:  [[0.9880929627462135, 0.9728250915394179], [0.9891371143801229, 0.9540567725774562], [0.9901625178749895, 0.9488898335585798]]
Recall for iterations:  [[0.9879999415700137, 0.9727711981399404], [0.9890370738262877, 0.9529535711690765], [0.9900742060825616, 0.9472498728474896]]


#### 5C NODE 3 10%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/4114672810.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C3trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3trim.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 212ms/step - loss: 0.5324 - accuracy: 0.7423 - val_loss: 1.7323 - val_accuracy: 0.0233
Epoch 2/5
16/16 [==============================] - 3s 182ms/step - loss: 0.1918 - accuracy: 0.9744 - val_loss: 3.9565 - val_accuracy: 0.1281
Epoch 3/5
16/16 [==============================] - 3s 176ms/step - loss: 0.0867 - accuracy: 0.9892 - val_loss: 3.8688 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0524 - accuracy: 0.9892 - val_loss: 1.6523 - val_accuracy: 0.1386
Epoch 5/5
16/16 [==============================] - 3s 163ms/step - loss: 0.0364 - accuracy: 0.9891 - val_loss: 1.2198 - val_accuracy: 0.2383
Epoch 1/5
14/14 [==============================] - 3s 184ms/step - loss: 0.0220 - accuracy: 0.9948 - val_loss: 0.8367 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 171ms/step - loss: 0.0195 - accuracy: 0.9950 - val_loss: 0.6023 - val_accuracy: 0.7283
Epoch 3/5
14/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07911981642246246, 0.10972201079130173], [0.11108022183179855, 0.12621992826461792], [0.10594591498374939, 0.14654435217380524]]
Accuracy for iterations:  [[0.9884600639343262, 0.9746421575546265], [0.989482581615448, 0.9660139679908752], [0.9902129769325256, 0.9479401111602783]]
F1 for iterations:  [[0.9884641874354215, 0.9746253145543056], [0.9894866514296236, 0.9659476699492876], [0.9902162674857102, 0.9476694014296102]]
Precision for iterations:  [[0.9885660727407686, 0.9746612204195951], [0.9896081802206068, 0.9662745375191877], [0.9903052955349512, 0.9495162369660378]]
Recall for iterations:  [[0.9884600777118817, 0.9746421565065756], [0.9894826024715884, 0.9660139504468502], [0.9902129772999503, 0.9479401293322677]]


#### 5C NODE 3 25%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/512859027.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C3trim.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3trim.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5393 - accuracy: 0.7489 - val_loss: 1.6758 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 186ms/step - loss: 0.2027 - accuracy: 0.9544 - val_loss: 3.6085 - val_accuracy: 0.1320
Epoch 3/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0847 - accuracy: 0.9892 - val_loss: 4.0488 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0519 - accuracy: 0.9894 - val_loss: 2.0923 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 206ms/step - loss: 0.0370 - accuracy: 0.9903 - val_loss: 1.6371 - val_accuracy: 0.1401
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0219 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 219ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.6223 - val_accuracy: 0.7294
Epoch 3/5

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3trim.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5393 - accuracy: 0.7489 - val_loss: 1.6758 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 186ms/step - loss: 0.2027 - accuracy: 0.9544 - val_loss: 3.6085 - val_accuracy: 0.1320
Epoch 3/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0847 - accuracy: 0.9892 - val_loss: 4.0488 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0519 - accuracy: 0.9894 - val_loss: 2.0923 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 206ms/step - loss: 0.0370 - accuracy: 0.9903 - val_loss: 1.6371 - val_accuracy: 0.1401
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0219 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 219ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.6223 - val_accuracy: 0.7294
Epoch 3/5

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08544706553220749, 0.12328588217496872], [0.09680859744548798, 0.13693860173225403], [0.12291232496500015, 0.15408408641815186]]
Accuracy for iterations:  [[0.9878538846969604, 0.9722262620925903], [0.9889128804206848, 0.9629804491996765], [0.9890589714050293, 0.9475768208503723]]
F1 for iterations:  [[0.9878578666501733, 0.9722010170737153], [0.988916709205785, 0.9628907034001225], [0.9890627905095888, 0.9472986746263102]]
Precision for iterations:  [[0.9879434565329552, 0.9722715522619876], [0.9890099785924518, 0.9633804123844554], [0.9891592171717783, 0.949208839513966]]
Recall for iterations:  [[0.9878538666043414, 0.9722262588098525], [0.9889129101054661, 0.9629804548426942], [0.9890589850711385, 0.9475768364455424]]
